# Data Warehouse Medicare Texas QA - Claim Diag

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

## Row Count and Claim Count

Similar to the member_enrollment_monthly table, the claim_header table has the row count should equal the number of unique claims.

For this table, we extract claim data from the s, o, and f tables from the mcrten schema. 


In [15]:
query = ''' drop table if exists qa_reporting.dw_mcrt_claim_diag_counts;
create table qa_reporting.dw_mcrt_claim_diag_counts
(
    calendar_year int,
    dw_row_count bigint,
    src_row_count bigint,
    row_count_diff bigint,
    row_count_percentage float,
    dw_uth_clm_id_count bigint,
    dw_src_clm_id_count bigint,
    src_clm_count bigint,
    clm_count_diff bigint,
    clm_count_percentage float,
    dw_uth_mbr_id_count bigint,
    dw_src_mbr_id_count bigint,
    src_mbr_count bigint,
    mbr_count_diff bigint,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [16]:
with connection.cursor() as cursor:
    query = f'''
    insert into qa_reporting.dw_mcrt_claim_diag_counts
    (calendar_year, dw_row_count, dw_uth_clm_id_count, date_generated)
    select year, count(*), count(distinct uth_claim_id), current_date
    from dw_staging.mcrt_claim_diag
    where diag_position = 1
    group by 1
    '''
    
    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_mcrt_claim_diag_counts b
    set dw_src_clm_id_count = count
    from (
        select year,  count(distinct claim_id_src) as count 
        from dw_staging.mcrt_claim_diag
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_mcrt_claim_diag_counts b
    set dw_uth_mbr_id_count = count
    from (
        select year, count(distinct uth_member_id) as count 
        from dw_staging.mcrt_claim_diag
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)
    
    query = f'''
    update qa_reporting.dw_mcrt_claim_diag_counts b
    set dw_src_mbr_id_count = count
    from (
        select year, count(distinct member_id_src) as count 
        from dw_staging.mcrt_claim_diag
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)

In [17]:
with connection.cursor() as cursor:
    query = '''
    with clms as (
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_texas.hha_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_texas.outpatient_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_texas.dme_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date)as year, bene_id, clm_id
        from medicare_texas.inpatient_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_texas.bcarrier_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_texas.hospice_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_texas.snf_base_claims_k
        where icd_dgns_cd1 is not null
    ),
    clm_counts as (
        select year, count(*) row_count, count(distinct bene_id) pat_count, count(distinct clm_id) clm_count
        from clms
        group by 1
    )
    update qa_reporting.dw_mcrt_claim_diag_counts a
    set src_row_count = b.row_count,
    row_count_diff = a.dw_row_count - b.row_count,
    row_count_percentage = 100. * abs(a.dw_row_count - b.row_count) / b.row_count,
    src_clm_count = b.clm_count,
    clm_count_diff = a.dw_uth_clm_id_count - b.clm_count,
    clm_count_percentage = 100. * abs(a.dw_uth_clm_id_count - b.clm_count) / b.clm_count,
    src_mbr_count = b.pat_count,
    mbr_count_diff = a.dw_uth_mbr_id_count - b.pat_count,
    mbr_count_percentage = 100. * abs(a.dw_uth_mbr_id_count - b.pat_count) / b.pat_count
    from clm_counts b
    where a.calendar_year = b.year
    ;
    '''

    cursor.execute(query)

In [18]:
query = '''select * from qa_reporting.dw_mcrt_claim_diag_counts;'''

df = pd.read_sql(query, con=connection)
df.sort_values(['clm_count_percentage'], ascending=False)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,calendar_year,dw_row_count,src_row_count,row_count_diff,row_count_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
1,2020,69154884,69288719.0,-133835.0,0.193156,69154884,69154884,69288719.0,-133835.0,0.193156,2383694,2383694,2384882.0,-1188.0,0.049814,2023-10-04
6,2014,78590794,78623020.0,-32226.0,0.040988,78590794,78590794,78623020.0,-32226.0,0.040988,2520193,2520193,2521006.0,-813.0,0.032249,2023-10-04
0,2018,78096488,78125243.0,-28755.0,0.036806,78096488,78096488,78125243.0,-28755.0,0.036806,2508827,2508827,2509887.0,-1060.0,0.042233,2023-10-04
12,2017,80780356,80799923.0,-19567.0,0.024217,80780356,80780356,80799923.0,-19567.0,0.024217,2549117,2549117,2547302.0,1815.0,0.071252,2023-10-04
8,2019,77333467,77349845.0,-16378.0,0.021174,77333467,77333467,77349845.0,-16378.0,0.021174,2453552,2453552,2453152.0,400.0,0.016306,2023-10-04
15,2016,80516454,80528127.0,-11673.0,0.014496,80516454,80516454,80528127.0,-11673.0,0.014496,2546813,2546813,2547595.0,-782.0,0.030696,2023-10-04
17,2015,79304963,79315887.0,-10924.0,0.013773,79304963,79304963,79315887.0,-10924.0,0.013773,2536400,2536400,2537197.0,-797.0,0.031413,2023-10-04
2,2002,7,NaN,NaN,NaN,7,7,NaN,NaN,NaN,7,7,NaN,NaN,NaN,2023-10-04
3,2008,2,NaN,NaN,NaN,2,2,NaN,NaN,NaN,2,2,NaN,NaN,NaN,2023-10-04
4,2003,1,NaN,NaN,NaN,1,1,NaN,NaN,NaN,1,1,NaN,NaN,NaN,2023-10-04


## Diagnosis Codes

Here we will check if we have valid ICD Diagnosis codes using our reference tables.

In [3]:
query = '''drop table if exists qa_reporting.dw_mcrt_diag_counts;
select year, diag_cd, count(*) as diag_count
into qa_reporting.dw_mcrt_diag_counts
from dw_staging.mcrt_claim_diag
group by 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [4]:
diag_cd_df = pd.read_sql('select * from qa_reporting.dw_mcrt_diag_counts;', con=connection)
diag_cd_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_cd,diag_count
0,2015,71661,692
1,2018,H35353,7184
2,2018,H33041,33
3,2016,S82202P,35
4,2017,F3012,573
...,...,...,...
309831,2014,3207,109
309832,2020,M84471P,10
309833,2015,R4701,15474
309834,2019,H1132,10080


We see that we have a lot of claims with diagnosis codes that are not in our reference list. If we take a further look at some of these codes, we see that in our reference list, there is at least one more digit missing from these codes. The missing digits helps specify the diagnosis code.

In [5]:
query = '''
select a.*
from qa_reporting.dw_mcrt_diag_counts a
left join reference_tables.ref_cms_icd_cm_codes b
on a.diag_cd = cd_value
where b.cd_value is null;
'''

invalid_diag_df = pd.read_sql(query, con=connection)
invalid_diag_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_cd,diag_count
0,2015,36382,1
1,2015,3460,1
2,2016,FO9,1
3,2015,J4411,2
4,2016,M5036,1
...,...,...,...
2595,2016,M1190,1
2596,2014,145,9
2597,2015,19023,1
2598,2015,7373,1


Overall, the number of invalid diagnosis codes is neglible compared to the overall number of diagnosis codes in the claim_diag table.

In [6]:
invalid_diag_df.groupby('year')['diag_count'].sum()

year
2014    1759
2015    1874
2016     414
2017     285
2018     260
2019     518
2020     136
Name: diag_count, dtype: int64

In [7]:
diag_comp_df = pd.DataFrame({'overall_diag_count': diag_cd_df.groupby('year')['diag_count'].sum(),
                            'invalid_diag_count': invalid_diag_df.groupby('year')['diag_count'].sum(),
                            'valid_diag_count': diag_cd_df.groupby('year')['diag_count'].sum() - invalid_diag_df.groupby('year')['diag_count'].sum()})
diag_comp_df.loc[diag_comp_df['valid_diag_count'].isna(),'valid_diag_count'] = diag_comp_df.loc[diag_comp_df['valid_diag_count'].isna(),'overall_diag_count']
diag_comp_df['valid_diag_count'] =  diag_comp_df['valid_diag_count'].astype(int)
diag_comp_df['invalid_to_valid_percent'] = 100. * diag_comp_df['invalid_diag_count'] / diag_comp_df['valid_diag_count']
diag_comp_df  

,overall_diag_count,invalid_diag_count,valid_diag_count,invalid_to_valid_percent
year,,,,
2001,74,NaN,74,NaN
2002,70,NaN,70,NaN
2003,9,NaN,9,NaN
2004,26,NaN,26,NaN
2005,53,NaN,53,NaN
2006,22,NaN,22,NaN
2007,57,NaN,57,NaN
2008,9,NaN,9,NaN
2009,62,NaN,62,NaN


In [8]:
invalid_diag_df.groupby('year').max()

,diag_cd,diag_count
year,,
2014,e9342,218
2015,m1712,185
2016,Z9982,28
2017,Z995,61
2018,ZB53,93
2019,ZOOOO,222
2020,Z99814,7


## Diagnosis Code Position

In [9]:
query = '''drop table if exists qa_reporting.dw_mcrt_diag_position;
select year, diag_position, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_mcrt_diag_position
from dw_staging.mcrt_claim_diag
group by 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(query)


In [10]:
diag_position_df = pd.read_sql('select * from qa_reporting.dw_mcrt_diag_position;', con=connection)
diag_position_df.sort_values(['year', 'diag_position'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_position,claim_count
18,2001,1,6
107,2001,2,5
345,2001,3,5
217,2001,4,5
228,2001,5,5
...,...,...,...
141,2020,21,500229
304,2020,22,431361
216,2020,23,370791
187,2020,24,316053


In [11]:
diag_position_df['diag_position'].unique()

array([ 5, 15, 19, 22, 12,  9, 13,  8,  7,  1,  4, 17, 18, 24, 10, 14,  3,
        2, 20, 21, 25, 11,  6, 16, 23], dtype=int64)

Checking if the counts for the diag_position are correct. The higher the diag_position is, the less counts there should be. If we sort the counts of the diag_position by year and assign their order, this value should match with the diag_position value.

In [12]:
diag_position_df['row_rank'] = diag_position_df.sort_values(['year', 'claim_count'], ascending=[True, False]).groupby(['year']).cumcount()+1
diag_position_df['position_check'] = diag_position_df['row_rank'] == diag_position_df['diag_position']
diag_position_df[~diag_position_df['position_check']]

,year,diag_position,claim_count,row_rank,position_check
0,2006,5,2,1,False
1,2004,15,1,3,False
3,2001,22,1,18,False
9,2010,13,8,12,False
12,2005,8,4,2,False
...,...,...,...,...,...
417,2005,7,4,10,False
418,2004,23,1,22,False
420,2009,4,5,9,False
423,2004,14,1,23,False


## ICD Version

In [13]:
query = '''
select year, icd_version, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_mcrt_icd_version_count
from dw_staging.mcrt_claim_diag
group by 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(query)

In [14]:
icd_version_df = pd.read_sql('select * from qa_reporting.dw_mcrt_icd_version_count;', con=connection)
icd_version_df.sort_values(['year', 'icd_version'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,icd_version,claim_count
32,2001,9,1
22,2001,None,5
1,2002,9,2
25,2002,None,5
14,2003,None,1
33,2004,9,2
10,2004,None,1
24,2005,9,2
3,2005,None,4
30,2006,None,2
